In [ ]:
!pip -q install wandb

!wget -q https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip -q nature_12K.zip
!rm nature_12K.zip
!mv ./inaturalist_12K/val ./inaturalist_12K/test

In [ ]:
import wandb
import os
os.environ["WANDB_SILENT"] = "true"
os.environ['WANDB_CONSOLE'] = 'off'
wandb.login()

In [ ]:
import torch
import numpy as np
import random

from hyperparams import partb_default_config, partb_config
from agent import Agent
from cnn import CNN
from utils import Utils
from guided_backprop import Guided_backprop

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEED = 123
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
train_path = "/content/inaturalist_12K/train"
test_path = "/content/inaturalist_12K/test"
BATCH_SIZE = 9
EPOCHS = 10
NUM_CLASSES = 10

def single_sweep():
    val_acc, test_acc = 0, 0
    wandb.init(config=partb_default_config)
    config = wandb.config

    HYPERPARAMS = config._as_dict()
    wandb.run.name = None
    wandb.run.name = HYPERPARAMS["model_name"][0] + "_strategy_" + HYPERPARAMS['strategy'] + "_data_aug_" + str(HYPERPARAMS['data_aug'])

    utils_agent = Utils(train_path, test_path, BATCH_SIZE, HYPERPARAMS["model_name"][1], train_transform=HYPERPARAMS['data_aug'])
    model = utils_agent.get_pretrained_models(HYPERPARAMS["model_name"][0], NUM_CLASSES, HYPERPARAMS['strategy']=='feature_extraction')
    agent = Agent(model, utils_agent, EPOCHS, is_inception=(HYPERPARAMS["model_name"][0]=='IV3'))
    val_acc = agent.train() # your model training code here
    _, test_acc = agent.evaluate(test_data=True, part_a=False)
    wandb.log({'val_acc': val_acc.cpu().item(), 'test_accuracy': test_acc.cpu().item()})
    wandb.finish(quiet=True)

# run the sweep
sweep_id = wandb.sweep(partb_config, project="test", entity="kbdl")
wandb.agent(sweep_id, function=single_sweep, count=2)